# [Day 5: Supply Stacks](https://adventofcode.com/2022/day/5)

## Part 1

In [1]:
import copy
import itertools
import pathlib
import re

In [2]:
test_input_data = """
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""

In [3]:
StackType = list[list[str]]
InstructionListType = list[tuple[int, ...]]

def add_layer_to_stacks(stacks: StackType, line: str) -> StackType:
    items = list(itertools.islice(line, 1, None, 4))
    while len(stacks) < len(items):
        stacks.append([])
    for i, item in enumerate(items):
        if item.strip():
            stacks[i].append(item)
    return stacks

def parse_input(data: str) -> tuple[StackType, InstructionListType]:
    re_instruction = re.compile(r"^\s*move\s*(\d+)\s*from\s*(\d+)\s*to\s*(\d+)\s*$")
    state = 0
    stacks: StackType = []
    instructions = []
    for line in data.splitlines():
        if (state == 0) and ("[" in line):
            state += 1
        if state == 1:
            if "[" in line:
                stacks = add_layer_to_stacks(stacks, line)
            else:
                state = 2
        if (state == 2) and (mt_instruction := re_instruction.match(line)):
            instructions.append(tuple(map(int, mt_instruction.groups())))
    return stacks, instructions

test_stacks, test_instructions = parse_input(test_input_data)
test_stacks, test_instructions

([['N', 'Z'], ['D', 'C', 'M'], ['P']],
 [(1, 2, 1), (3, 1, 3), (2, 2, 1), (1, 1, 2)])

In [4]:
def move_crates(stacks: StackType, instructions: InstructionListType) -> StackType:
    stacks = copy.deepcopy(stacks)  # do not modify start position
    for n_crates, stack_from, stack_to in instructions:
        stack_from -= 1
        stack_to -= 1
        while n_crates > 0:
            crane = stacks[stack_from].pop(0)
            stacks[stack_to].insert(0, crane)
            n_crates -= 1
    return stacks

def get_top_crates(stacks):
    # account for empty stacks
    return "".join([stack[0] if stack else " " for stack in stacks])

In [5]:
%%time
get_top_crates(move_crates(test_stacks, test_instructions)) == "CMZ"

CPU times: total: 0 ns
Wall time: 0 ns


True

In [6]:
stacks, instructions = parse_input(pathlib.Path("../data/day05.txt").read_text())
len(stacks) == 9, instructions[-1] == (1, 1, 9)

(True, True)

In [7]:
%%time
print(f"Answer part 1: {get_top_crates(move_crates(stacks, instructions))}\n")

Answer part 1: ZWHVFWQWW

CPU times: total: 0 ns
Wall time: 3 ms


## Part 2

In [8]:
def move_crates_cm9001(stacks: StackType, instructions: InstructionListType) -> StackType:
    stacks = copy.deepcopy(stacks)  # do not modify start position
    for n_crates, stack_from, stack_to in instructions:
        stack_from -= 1
        stack_to -= 1
        crane = stacks[stack_from][:n_crates]
        stacks[stack_from] = stacks[stack_from][n_crates:]
        crane.extend(stacks[stack_to])
        stacks[stack_to] = crane
    return stacks

In [9]:
%%time
get_top_crates(move_crates_cm9001(test_stacks, test_instructions)) == "MCD"

CPU times: total: 0 ns
Wall time: 998 µs


True

In [10]:
%%time
print(f"Answer part 2: {get_top_crates(move_crates_cm9001(stacks, instructions))}\n")

Answer part 2: HZFZCCWWV

CPU times: total: 0 ns
Wall time: 998 µs
